In [ ]:
# Iman Wahle
# Test multi-scale CNN 
# November 2019

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout
from keras.losses import sparse_categorical_crossentropy
from keras.optimizers import SGD
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard


Using TensorFlow backend.
/Users/imanwahle/anaconda2/lib/python2.7/site-packages/cryptography/hazmat/primitives/constant_time.py:26: CryptographyDeprecationWarning: Support for your Python version is deprecated. The next version of cryptography will remove support. Please upgrade to a release (2.7.7+) that supports hmac.compare_digest as soon as possible.
  utils.PersistentlyDeprecated2018,


In [2]:
X_data = np.load("X_data_fits.npy")
y_data = np.load("y_data_fits.npy")

In [3]:
f, ax = plt.subplots(1, 3, figsize=(18,4))

y2d = y_data.copy()
y2d[:,0] = y_data[:,0]//20
idx = np.where(y2d[:,0]==5)[0]
y2d[idx,0] = 4
y2d[:,1] = y_data[:,1]//20
y = y2d[:,1]*5 + y2d[:,0]

# ax[0].plot(y_data[:,0],y2d[:,0],'.')
# ax[0].set_xlabel('spin')
# ax[0].set_ylabel('spin class')
# ax[1].plot(y_data[:,1],y2d[:,1],'.')
# ax[1].set_xlabel('incl')
# ax[1].set_ylabel('incl class')
# sns.scatterplot(y_data[:,0], y_data[:,1], hue=y, palette="jet",ax=ax[2])

In [4]:
# format data

# normalize the map to [0,1]
maxmap_arr = np.max(X_data.reshape(X_data.shape[0],-1), axis = 1)
Xnorm = np.tile(maxmap_arr, (X_data.shape[1], X_data.shape[2], 1))
Xnorm = np.swapaxes(Xnorm, 0, 2)
X = X_data/Xnorm

X = X[..., np.newaxis]
n_classes = len(np.unique(y))

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# one hot
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

In [ ]:
# # original single-scale model definition

# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3),
#                  activation='relu',
#                  input_shape=(100,100,1)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(n_classes, activation='softmax'))

# model.summary()


In [22]:
# model 1 definition
model_name = "model1"
model = Sequential()
model.add(Conv2D(32, kernel_size=(5,5),
                 activation='relu',
                 input_shape=(100,100,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

model.summary()

# training
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy', 'mse'])

# model.summary()
batch_size = 32
epochs = 10
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[TensorBoard(log_dir='/tmp/multiscale/' + model_name)])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Save the model
model_yaml = model.to_yaml()
with open('multiscale_' + model_name + '_weights.yaml', "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights('multiscale_' + model_name + '_weights.h5')
yaml_file.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 96, 96, 32)        832       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 94, 94, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 47, 47, 64)        0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 47, 47, 64)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 141376)            0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               18096256  
_________________________________________________________________
dropout_16 (Dropout)         (None, 128)               0         
__________

In [24]:
# model 2 definition
model_name = "model2"
model = Sequential()
model.add(Conv2D(32, kernel_size=(5,5),
                 activation='relu',
                 input_shape=(100,100,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

model.summary()

# training
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy', 'mse'])

# model.summary()
batch_size = 32
epochs = 10
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[TensorBoard(log_dir='/tmp/multiscale/' + model_name)])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Save the model
model_yaml = model.to_yaml()
with open('multiscale_' + model_name + '_weights.yaml', "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights('multiscale_' + model_name + '_weights.h5')
yaml_file.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 96, 96, 32)        832       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 46, 46, 64)        18496     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 23, 23, 64)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 23, 23, 64)        0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 33856)             0         
_________________________________________________________________
dense_17 (Dense)             (None, 128)               4333696   
__________

In [25]:
# model 3 definition
model_name = "model3"
model = Sequential()
model.add(Conv2D(32, kernel_size=(7,7),
                 activation='relu',
                 input_shape=(100,100,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

model.summary()

# training
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy', 'mse'])

# model.summary()
batch_size = 32
epochs = 10
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[TensorBoard(log_dir='/tmp/multiscale/' + model_name)])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Save the model
model_yaml = model.to_yaml()
with open('multiscale_' + model_name + '_weights.yaml', "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights('multiscale_' + model_name + '_weights.h5')
yaml_file.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 94, 94, 32)        1600      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 47, 47, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 45, 45, 64)        18496     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 22, 22, 64)        0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 30976)             0         
_________________________________________________________________
dense_19 (Dense)             (None, 128)               3965056   
__________

In [21]:
# model 4 definition
model_name = "model4"
model = Sequential()
model.add(Conv2D(32, kernel_size=(5,5),
                 activation='relu',
                 input_shape=(100,100,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

model.summary()

# training
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy', 'mse'])

# model.summary()
batch_size = 32
epochs = 50
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[TensorBoard(log_dir='/tmp/multiscale/' + model_name)])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Save the model
model_yaml = model.to_yaml()
with open('multiscale_' + model_name + '_weights.yaml', "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights('multiscale_' + model_name + '_weights.h5')
yaml_file.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 96, 96, 32)        832       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 46, 46, 64)        18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 23, 23, 64)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 23, 23, 64)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 33856)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               4333696   
__________

In [ ]:
# model 5 definition
model_name = "model5"
model = Sequential()
model.add(Conv2D(32, kernel_size=(7,7),
                 activation='relu',
                 input_shape=(100,100,1)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

model.summary()

# training
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy', 'mse'])

# model.summary()
batch_size = 32
epochs = 10
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[TensorBoard(log_dir='/tmp/multiscale/' + model_name)])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Save the model
model_yaml = model.to_yaml()
with open('multiscale_' + model_name + '_weights.yaml', "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights('multiscale_' + model_name + '_weights.h5')
yaml_file.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 94, 94, 32)        1600      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 90, 90, 64)        51264     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 88, 88, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 44, 44, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 44, 44, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 123904)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               15859840  
__________

In [ ]:
# model 6 definition
model_name = "model6"
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),
                 activation='relu',
                 input_shape=(100,100,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

model.summary()

# training
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy', 'mse'])

# model.summary()
batch_size = 32
epochs = 100
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[TensorBoard(log_dir='/tmp/multiscale/' + model_name)])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Save the model
model_yaml = model.to_yaml()
with open('multiscale_' + model_name + '_weights.yaml', "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights('multiscale_' + model_name + '_weights.h5')
yaml_file.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 98, 98, 32)        320       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 96, 96, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 48, 48, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 147456)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               18874496  
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
__________

6817/6817 [==============================] - 369s 54ms/step - loss: 0.2849 - acc: 0.8866 - mean_squared_error: 0.0066 - val_loss: 0.2533 - val_acc: 0.9054 - val_mean_squared_error: 0.0056
Epoch 35/100
6817/6817 [==============================] - 369s 54ms/step - loss: 0.2887 - acc: 0.8837 - mean_squared_error: 0.0066 - val_loss: 0.1263 - val_acc: 0.9490 - val_mean_squared_error: 0.0030
Epoch 36/100
6817/6817 [==============================] - 370s 54ms/step - loss: 0.2760 - acc: 0.8873 - mean_squared_error: 0.0064 - val_loss: 0.3486 - val_acc: 0.8869 - val_mean_squared_error: 0.0072
Epoch 37/100
6817/6817 [==============================] - 369s 54ms/step - loss: 0.2845 - acc: 0.8895 - mean_squared_error: 0.0065 - val_loss: 0.1568 - val_acc: 0.9327 - val_mean_squared_error: 0.0038
Epoch 38/100
6817/6817 [==============================] - 371s 54ms/step - loss: 0.2774 - acc: 0.8884 - mean_squared_error: 0.0065 - val_loss: 0.1112 - val_acc: 0.9538 - val_mean_squared_error: 0.0026
Epoch 39

In [ ]:
# model 7 definition
model_name = "model7"
model = Sequential()
model.add(Conv2D(32, kernel_size=(5,5),
                 activation='relu',
                 input_shape=(100,100,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

model.summary()

# training
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy', 'mse'])

# model.summary()
batch_size = 32
epochs = 100
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[TensorBoard(log_dir='/tmp/multiscale/' + model_name)])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Save the model
model_yaml = model.to_yaml()
with open('multiscale_' + model_name + '_weights.yaml', "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights('multiscale_' + model_name + '_weights.h5')
yaml_file.close()